In [18]:
# import all the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict,OrderedDict
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_distances
from operator import itemgetter
from scipy.spatial.distance import cosine
import json
import pickle

In [10]:
# reading data from dataset
df = pd.read_csv('combine.csv')
# breif about the columns in the dataset
df.head()

,id,phoneNo,location,occupation,certification,age,gender,type,availabilityPreference,availability,minimumWage,experience,clientsAttended,aadharCard,doorstepService,references,liscenced,shoppingliscence
0,1,9326523642,Ambegaon,Accountant,true,26,Female,Full Time,Afternoon,Whole Day,9500,8,49,065660562714,no,false,false,true
1,2,7029738484,Ambegaon,Brick mason,true,36,Male,Part Time,Morning,Morning,4000,4,25,394728990924,no,false,false,true
2,3,8386070476,Ambegaon,Carpenter,true,31,Male,Part Time,Evening,Evening,5500,8,42,777556713881,yes,true,false,false
3,4,8208265762,Ambegaon,Cashier,false,20,Female,Part Time,Evening,Evening,2500,2,13,938143072796,no,true,false,true
4,5,9127458171,Ambegaon,Childcare Worker,true,33,Female,Full Time,Evening,Whole Day,2000,3,28,112614367501,yes,true,false,true


In [11]:
df = df.drop(['phoneNo','id','availabilityPreference','aadharCard','liscenced','shoppingliscence'],axis=1)
df.dropna(inplace=True)
# df.head()

In [12]:
# l = preprocessing.LabelEncoder()
# nan_rows = df[df['location'].isnull()]
# nan_rows
# creating classes for dummy purpose only
#----------------------------------------------------------------------------------------------------------------------
def classes_maker():
    for i in df.columns:
        le = preprocessing.LabelEncoder()
        le.fit(df[i])
        df[i] = le.transform(df[i])
classes_maker()
# df.head()

In [13]:
# le.fit(df['certification'])
# df['certification'] = le.transform(df['certification'])
# le.fit(df['occupation'])
# df['occupation'] = le.transform(df['occupation'])
# le.fit(df['gender'])
# df['gender'] = le.transform(df['gender'])
# le.fit(df['type'])
# df['type'] = le.transform(df['type'])
# le.fit(df['availability'])
# df['availability'] = le.transform(df['availability'])
# le.fit(df['doorstepService '])
# df['doorstepService '] = le.transform(df['doorstepService '])
# le.fit(df['references'])
# df['references'] = le.transform(df['references'])
# le.fit(df['clientsAttended'])
# df['clientsAttended'] = le.transform(df['clientsAttended'])
# le.fit(df['experience'])
# df['experience'] = le.transform(df['experience'])
# df.head()
# np.shape(df)[0]

In [29]:
occupations=defaultdict(list)
# location: index and occupation where index is the position of row in Dataframe
def all_occupations_in_a_location():
    for index,row in df.iterrows():
        occupations[row['location']].append((index,row['occupation']))
        
    for key, values in occupations.items():
        t_set = set(occupations.get(key))
        occupations[key] = list(t_set)
all_occupations_in_a_location()
# print(occupations.get(0))
# print(sorted(occupations.items()))
# print(occupations.get(0)[2])
# print(occupations[0][5])
# df.head()
# temp = occupations[0]
# temp.sort(key=itemgetter(1))
# print(temp)
# print(temp[0][1])
# indexes = []
# print(len(occupations[0]))
# for i in range(len(occupations.get(0))):
#     if i+1 < len(occupations.get(0)):
#         if temp[i][1] == temp[i+1][1]:
#             indexes.append((temp[i+1][0],temp[i][0]))
# print(indexes)
# ----------------------------------------------------------------------------------------------------
# to unpickle use 
# from ast import literal_eval
# with open(path,'r') as f:
# ...     type(pickle.loads(literal_eval(f.read())))

with open('occupations.txt','w') as f:
    f.write(str(pickle.dumps(occupations)))
print("Occupation dict has been pickled.")

Occupation dict has been pickled.


In [30]:
dist = np.ndarray(shape=(np.shape(df)[0],np.shape(df)[0]))
def cos_func(index1,index2):
    # will calcuate the cosine distance between 2 workers and return to helper
    #print(index1,index2)
    return cosine_distances(df.loc[index1].values.reshape(1, -1),df.loc[index2].values.reshape(1, -1))
                            
def cos_helper(indexes):
    # worker with similar occupation in a given location is passed as 'indexes'
    #TODO: pass pair of tuples in the 'indexes' list to the cosine distance function which will return the distance between
    # those 2 data points and will be saved in 'dist' list w.r.t index of those points so that we have distance of all workers 
    # which we will use to make a matrix/ to cluster
    for i in indexes:
        for j in indexes:
            if i != j:
                dist[i][j] = cos_func(i,j)

# find workers at a location having a specific occupation for calculating cosine distance between them
def commonities():
        for k, v in occupations.items(): # location iterator
            type(occupations[k]) # list type
            temp_list = occupations[k]
            indexes = []
            temp_list.sort(key=itemgetter(1)) # sort list according to occupation
            # print(temp_list)
                        
            # TODO: Find similar occupations and then pass it to cosine_distance calculator
                #print(occupations[k][l]) #tuple type
                
                # this will find all workers having similar occupation in a particular location
            for i in range(len(occupations[k])):
                if i+1 < len(occupations[k]):
                    if temp_list[i][1] == temp_list[i+1][1]:
                        indexes.append(temp_list[i][0])
            #print(df.loc[indexes[4]],df.loc[indexes[1]])
            #print(len(indexes)) total number of workers with similar occupations in a given location
            cos_helper(indexes)
                
commonities()
# print(dist)
#-----------------------------------------------------------------------------------------------
# TO UNPICKLE
# with open(path,'r') as f:
# ...     type(pickle.loads(literal_eval(f.read())))

# idea is to search for the similar occupation and return its index and remove it from the list as there might be duplicates

# return index,array
# def cos_dist():
#     for key in occupations.keys():
#         for l in len():
#             dist[index].append(cosine_distances(occupations[key].reshape(1,-1),row.reshape(1,-1))))

MemoryError: 

In [35]:
temp_dist = pickle.dumps(dist)

MemoryError: 

In [ ]:
plt.plot(dist)
plt.show()